In [1]:
from google.cloud import speech_v1p1beta1 as speech
from google.cloud import storage
from pydub import AudioSegment

import json
client = speech.SpeechClient()

In [2]:
gcs_uri = "gs://ami_corpus/amicorpus/ES2002a/audio/ES2002a.Mix-Headset.wav"

In [3]:
audio = speech.RecognitionAudio(uri=gcs_uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="en-US",
    enable_speaker_diarization=True,
    diarization_speaker_count=4,
)

In [4]:
print("Waiting for operation to complete...")
response = client.long_running_recognize(config=config, audio=audio)

Waiting for operation to complete...


In [5]:
result = response.result().results

In [7]:
words_info = result[-1].alternatives[0].words

In [14]:
def get_speaker_timings(words, speaker_count):
    timing = [0] * speaker_count
    for word in words: 
        timing[word.speaker_tag-1] += (word.end_time - word.start_time).total_seconds() 
    return [time / 60 for time in timing]

def get_speaker_stats(timing):
    return np.mean(timing), np.std(timing), np.var(timing)

def get_benchmark_communication():
    pass
timings = get_speaker_timings(words_info, 4)
get_speaker_stats(timings)

[4.941666666666662,
 2.0066666666666655,
 5.440000000000003,
 0.013333333333333334]

In [15]:

old_speaker = ""
old_end = 0
start = 0
analysis_data = []
transcript = []

for word in words_info:
    tag = word.speaker_tag
    print(word.word, word.speaker_tag)
    if(tag != old_speaker):
        analysis_data.append([tag, start, old_end, transcript])
        start = word.start_time
        transcript = []
    else: 
        transcript.append(word.word)
    old_end = word.end_time
    old_speaker = tag


NameError: name 'result' is not defined

In [21]:
print(analysis_data)

[[1, 0, 0, []], [3, datetime.timedelta(seconds=3, microseconds=700000), datetime.timedelta(seconds=6, microseconds=800000), ['God', 'she', 'produced', 'a']], [4, datetime.timedelta(seconds=6, microseconds=800000), datetime.timedelta(seconds=9, microseconds=400000), ['presentation', 'I', 'think', "it's", 'already', 'on', 'actually']], [3, datetime.timedelta(seconds=14, microseconds=900000), datetime.timedelta(seconds=34, microseconds=900000), ['kind', 'of', 'medicine', 'work', 'a', 'place', 'in', 'the', 'back', 'but']], [4, datetime.timedelta(seconds=37, microseconds=900000), datetime.timedelta(seconds=40, microseconds=800000), ['right']], [3, datetime.timedelta(seconds=50, microseconds=200000), datetime.timedelta(seconds=50, microseconds=800000), []], [1, datetime.timedelta(seconds=55, microseconds=800000), datetime.timedelta(seconds=74, microseconds=500000), ["it's", 'the', 'kickoff', 'meeting', 'for', 'our', 'project', 'this', 'is', 'just', 'what', "we're", 'going', 'to', 'be', 'doin

In [29]:
response.result()

TypeError: 'LongRunningRecognizeResponse' object is not subscriptable

In [14]:
speaker_change_times = [x[1].seconds * 1000 + x[1].microseconds  / 1000 for x in analysis_data[1:]]

NameError: name 'analysis_data' is not defined

In [4]:
storage_client = storage.Client()
bucket = storage_client.get_bucket("ami_corpus")
blob = bucket.blob("amicorpus/ES2002a/audio/ES2002a.Mix-Headset.wav")
blob.download_to_filename("/tmp/headset.wav")

#Convert downloaded object and save the export to a tmp file
sound = AudioSegment.from_wav("/tmp/headset.wav")


In [24]:
sound = AudioSegment.from_wav("interruptions1.wav")

In [12]:
sound[1000:2000].export('newSong.wav', format="wav")

<_io.BufferedRandom name='newSong.wav'>

In [25]:
for time in range(1000, 55000, 1000): #speaker_change_times:
    audio_chunk=sound[time-500:time+500]
    audio_chunk.export( "audio_chunk_{}.wav".format(time/1000), format="wav")


In [5]:
sound.duration_seconds

1272.64

In [12]:


split_centers = get_split_centers(sound.duration_seconds)

In [13]:
len(split_centers)

255

In [5]:
import jxmlease

In [34]:
xml_files = ['ES2002a.A.words.xml', 'ES2002a.B.words.xml', 'ES2002a.C.words.xml', 'ES2002a.D.words.xml']

# THIS SHIT IS IMPORTANT 

In [65]:
def get_speaker_change_times(xml_files):
    speaker_change_times = []
    
    for xml_file in xml_files:
        with open(xml_file) as topic:
            topic_content = topic.read()

        root = jxmlease.parse(topic_content)
        words = root['nite:root']['w']

        for i in range(1, len(words)):
            if(words[i-1].get_xml_attr('endtime') != words[i].get_xml_attr('starttime')):
                speaker_change_times.append(words[i].get_xml_attr('starttime'))
    
    speaker_change_times = set(speaker_change_times)
        
    return [int(float(x)*1000) for x in speaker_change_times]
        
            

In [66]:
speaker_change_times = get_speaker_change_times(xml_files)
get_sound_chunks(["/tmp/headset.wav"], speaker_change_times)